In [1]:
#Importing the required libraries
import numpy as np
from numpy import random
import scipy.stats as stats
from drawFromADist import drawFromADist
import matplotlib.pyplot as plt
from scipy.linalg import norm

In [2]:
def find(a) :
    '''This function is written to convert the matlab code into python'''
    return np.array([i for i,val in enumerate(a) if val > 0])

In [3]:
def generate_gaussian(mean=[0,1],sigma=[0.2,0.2],plot=False) :
    '''returns a randomly chosen point from a gaussian distribution'''
    if plot :
        x = np.linspace(-1,3,1000)
        pdf = np.zeros(shape=x.shape)
        y=np.array([])
        for m,s in zip(mean,sigma) :
            pdf += stats.norm.pdf(x, m, s)
        plt.plot(x, pdf)
    
    choice=np.random.choice(mean)
    return np.random.normal(mean[choice], sigma[choice])    

In [4]:
def temporary_context_model(schedule,beta_param=0.95) :
    N_WORLD_FEATURES = 5
    N_ITEMS = 10
    ENCODING_TIME = 500
    TEST_TIME = 20
    
    
    #schedule = np.sort(np.random.randint(1, ENCODING_TIME+1, size=N_ITEMS))
    schedule_load = ENCODING_TIME/np.median(np.diff(schedule,1))                
    encoding = np.zeros((N_ITEMS,N_WORLD_FEATURES+1))
    
    world_m = random.randint(1,10, size=(N_WORLD_FEATURES))       #Generating the world randomly       
    world_var = 1
    #delta = 0.05                       #what does this parameter affect?
    #beta_param = 0.001                #what does this parameter affect?
    m = 0 
    
    
    #simulating encoding

    for time in range(1,ENCODING_TIME+1) :
        delta=generate_gaussian()  #sampling the rate of drift over time (delta) 
        world_m = beta_param*world_m + delta
        world = random.normal(world_m, world_var)
        #any item I want to encode in memory, I encode in association with the
        #state of the world at that time.
        if m < N_ITEMS :
            if(time==schedule[m]):
                encoding[m,0:5] =  world
                encoding[m,5]=m             #encoding world_context and item
                m =  m + 1      
                
    out=np.zeros(TEST_TIME)
    while(time<ENCODING_TIME+TEST_TIME):
        #the state of the world is the retrieval cue
        world_m = beta_param*world_m + delta
        world = random.normal(world_m, world_var)
        soa=np.zeros(N_ITEMS)
        for m in range(N_ITEMS):
            t=world
            t=np.append(t,m)
            soa[m]=np.dot(encoding[m],t.T)
            #print(soa)
        soa=soa/norm(soa)
        out[time-ENCODING_TIME] = find(drawFromADist(soa))
        time = time + 1
        
    return len(np.unique(out)),schedule_load 

#### Shoving at the end :

In [5]:
schedule = np.arange(10)
schedule = schedule + 491
out = []
for i in range(50):
    success,schedule_load=temporary_context_model(schedule,beta_param=0.95)
    out.append(success)
print('Mean Successful retrivals over multiple trials:'+ str(np.mean(out)))
print('Schedule Load: '+str(schedule_load))

Mean Successful retrivals over multiple trials:8.5
Schedule Load: 500.0


Shoving at the end increases the retrival rate with an increase in schedule load too.

#### Finding the optimal schedule pattern :
- Looking at the expression of schedule load, we can see that the numerator is constant and the denominator is schedule dependent.
- In order to minimize the schedule load, we need to maximize the denominator. i.e the median of the diff array.
- An example diff array with maxmium median is [99, 99, 99, 99, 99,  1,  1,  1,  1]. So, an example schedule can be [1, 100, 199, 298, 397, 496, 497, 498, 499, 500].

In [6]:
schedule = np.array([1, 100, 199, 298, 397, 496, 497, 498, 499, 500])
out = []
for i in range(50):
    success,schedule_load=temporary_context_model(schedule,beta_param=0.95)
    out.append(success)
print('Mean Successful retrivals over multiple trials:'+ str(np.mean(out)))
print('Schedule Load: '+str(schedule_load))

Mean Successful retrivals over multiple trials:8.58
Schedule Load: 5.05050505050505
